### System settings verification

In [6]:
import sys
sys.version

'3.6.5 | packaged by conda-forge | (default, Apr  6 2018, 13:44:09) \n[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]'

In [7]:
!which python
!echo $PYTHONPATH
!java --version

/Users/harjeet/DevEnv/miniconda3/bin/python

java 10.0.1 2018-04-17
Java(TM) SE Runtime Environment 18.3 (build 10.0.1+10)
Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.1+10, mixed mode)


# Creating a Spark Context

In [8]:
from pyspark.sql import DataFrameReader, SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [9]:
spark = SparkSession\
    .builder\
    .config("master","local[*]")\
    .appName("LastFM")\
    .getOrCreate()

In [10]:
spark.sparkContext._conf.getAll()

[('spark.driver.host', '172.16.10.66'),
 ('master', 'local[*]'),
 ('spark.driver.port', '62331'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1526486605072'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'LastFM')]

# Getting the data
Here's the [README](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-readme.txt) for a description.
Briefly:
- 92834 "scrobbles" (play counts) per user of an artist
- 1892 users
- 17632 artists
- density/sparsity of the data: 0.28%

In [11]:
import os
import wget
url = 'http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip'
filename = wget.download(url)
filename

'hetrec2011-lastfm-2k.zip'

In [12]:
import zipfile
with zipfile.ZipFile(filename,"r") as zip_ref:
    zip_ref.extractall("./data")
!ls ./data

artists.dat                       user_friends.dat
readme.txt                        user_taggedartists-timestamps.dat
tags.dat                          user_taggedartists.dat
user_artists.dat


In [15]:
import glob
files_to_process = [os.path.relpath(dat_path, './data') for dat_path in glob.glob('./data/*.dat')]
files_to_process

['user_friends.dat',
 'user_taggedartists-timestamps.dat',
 'artists.dat',
 'tags.dat',
 'user_taggedartists.dat',
 'user_artists.dat']

In [ ]:
# Work in progress
def read_dats(filename):
    df  = spark.read(filename)

In [ ]:
dist_files_to_process = sc.parallelize(files_to_process)
processed = dist_files_to_process.map(lambda file: read_dats(file))
collected = processed.collect()